In [1]:
### Module 11 Assignment 3 Carrie Gold
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers==4.39.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from pathlib import Path

def download_file(url, save_path):
    """Download a file from a URL and save it to the specified path."""
    response = requests.get(url)
    response.raise_for_status()  # Check if the download was successful

    with open(save_path, 'wb') as f:
        f.write(response.content)
    print(f"File downloaded successfully: {save_path}")

# URL of the dataset
url_train = "https://github.com/UUDeCART/decart_rule_based_nlp/raw/master/data/training_v2.zip"
# Path where you want to save the downloaded file
save_path_train = Path("training_v2.zip")

download_file(url_train, save_path_train)

File downloaded successfully: training_v2.zip


In [4]:
url_test = "https://github.com/UUDeCART/decart_rule_based_nlp/raw/master/data/test_v2.zip"
save_path_test = Path("test_v2.zip")

download_file(url_test, save_path_test)

File downloaded successfully: test_v2.zip


In [5]:
import zipfile

zip_path = save_path_test
extract_path = save_path_test.parent / "test_v2"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"File unzipped successfully to: {extract_path}")
import zipfile

zip_path = "training_v2.zip"
extract_path = "training_v2"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"File unzipped successfully to: {extract_path}")

File unzipped successfully to: test_v2
File unzipped successfully to: training_v2


In [6]:
pip install quicksectx

Note: you may need to restart the kernel to use updated packages.


In [7]:
#pip install spacy

In [8]:
pip install git+https://github.com/medspacy/medspacy_io

  Cloning https://github.com/medspacy/medspacy_io to c:\users\carri\appdata\local\temp\pip-req-build-zyqgneq4
  Resolved https://github.com/medspacy/medspacy_io to commit 52682d9b208cafb65733ac2aef9d6cd3f139df63
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/medspacy/medspacy_io 'C:\Users\carri\AppData\Local\Temp\pip-req-build-zyqgneq4'


In [9]:
from spacy.lang.en import English
from medspacy_io.reader import BratDocReader
from medspacy_io.reader import BratDirReader
import spacy
from pathlib import Path
from medspacy_io.vectorizer import Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [10]:
# The dataset files does not include schema configuration, let's create one
concepts=['EVIDENCE_OF_PNEUMONIA', 'PNEUMONIA_DOC_NO', 'PNEUMONIA_DOC_YES']
lines=['[entities]']+concepts
Path('annotation.conf').write_text('\n'.join(lines))

67

In [11]:
#!python -m spacy download en_core_web_sm

In [12]:
# set up the Brat reader
nlp=spacy.load("en_core_web_sm", disable=['ner'])
dir_reader = BratDirReader(nlp=nlp, support_overlap=True, recursive=True, schema_file='annotation.conf')

found annotation.conf file


In [13]:
# This function will read brat annotation files and convert the snippet annotation into sentence labelled dataframe
def convert2df(data_folder):
  # read brat annotation into spaCy doc object.
  docs = dir_reader.read(txt_dir=data_folder)
  # convert snippet label into sentence-level labels and generate pandas dataframe
  df = Vectorizer.docs_to_sents_df(docs, track_doc_name=True, sent_window=2)
  # remove document-level labels
  df=df[~df['y'].str.contains('_DOC_')]
  return df[['X','y']]

In [14]:
train_df=convert2df('training_v2')
test_df=convert2df('test_v2')

In [15]:
def to_attr_classify(df:pd.DataFrame):
    keywords = ['opacity', 'infiltrate', 'pneumonia', 'effusion', 'consolidation']

    # Function to find offsets of keywords in sentences
    def find_offsets(sentence, keywords):
        offsets = []
        for keyword in keywords:
            start = sentence.find(keyword)
            if start != -1:
                end = start + len(keyword)
                offsets.append((keyword, start, end))
        return offsets

    # Apply the function and expand the list of offsets into a new dataframe
    rows = []
    for i, row in train_df.iterrows():
        offsets = find_offsets(row['X'].lower(), keywords)
        for keyword, start, end in offsets:
            rows.append({'X': row['X'], 'Keyword': keyword, 'Start': start, 'End': end, 'y':row['y']})

    # Creating a new dataframe from the rows with keywords found
    filtered_df = pd.DataFrame(rows)
    filtered_df['y']=filtered_df['y'].map(lambda x: 'NotAffirmed' if x == 'NEG' else 'Affirmed')
    return filtered_df

In [16]:
train_adf=to_attr_classify(train_df)

In [17]:
### Function defined with train_df file, so changed test_df to point to the same data set 
def to_attr_classify(df:pd.DataFrame):
    keywords = ['opacity', 'infiltrate', 'pneumonia', 'effusion', 'consolidation']

    # Function to find offsets of keywords in sentences
    def find_offsets(sentence, keywords):
        offsets = []
        for keyword in keywords:
            start = sentence.find(keyword)
            if start != -1:
                end = start + len(keyword)
                offsets.append((keyword, start, end))
        return offsets

    # Apply the function and expand the list of offsets into a new dataframe
    rows = []
    for i, row in test_df.iterrows():
        offsets = find_offsets(row['X'].lower(), keywords)
        for keyword, start, end in offsets:
            rows.append({'X': row['X'], 'Keyword': keyword, 'Start': start, 'End': end, 'y':row['y']})

    # Creating a new dataframe from the rows with keywords found
    filtered_df = pd.DataFrame(rows)
    filtered_df['y']=filtered_df['y'].map(lambda x: 'NotAffirmed' if x == 'NEG' else 'Affirmed')
    return filtered_df

In [18]:
test_adf=to_attr_classify(test_df)

In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
from datasets import Dataset

In [20]:
train_adf_dataset = Dataset.from_pandas(train_adf)
test_adf_dataset = Dataset.from_pandas(test_adf)

In [21]:
### Add a column with Not/Affirmed labels
train_adf_dataset = train_adf_dataset.map(lambda example: {'labels': 1 if example['y'] == 'Affirmed' else 0})

Map:   0%|          | 0/407 [00:00<?, ? examples/s]

In [22]:
test_adf_dataset = test_adf_dataset.map(lambda example: {'labels': 1 if example['y'] == 'Affirmed' else 0})

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

In [24]:
def preprocess_function(examples):
    return tokenizer(examples["X"], truncation=True)

In [25]:
train_adf_tok = train_adf_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/407 [00:00<?, ? examples/s]

In [26]:
test_adf_tok = test_adf_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

In [27]:
import tensorflow as tf

2024-03-24 22:39:53,529 tensorflow   WARNING  From c:\Users\carri\anaconda3\envs\bmi.biobert_env\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



In [28]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [29]:
id2label = {0: "NotAffirmed", 1: "Affirmed"}
label2id = {"NotAffirmed": 0, "Affirmed": 1}

In [30]:
from transformers import create_optimizer
from transformers import TFAutoModelForSequenceClassification

In [31]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(train_adf_tok) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [32]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

2024-03-24 22:40:24,553 tensorflow   WARNING  From c:\Users\carri\anaconda3\envs\bmi.biobert_env\Lib\site-packages\keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or

In [33]:
tf_train_set = model.prepare_tf_dataset(
    train_adf_tok,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [34]:
tf_validation_set = model.prepare_tf_dataset(
    test_adf_tok,
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [35]:
model.compile(optimizer=optimizer)  

In [36]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)

Epoch 1/3


2024-03-24 22:40:44,223 tensorflow   WARNING  From c:\Users\carri\anaconda3\envs\bmi.biobert_env\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.



25/25 [==============================] - 175s 6s/step - loss: 0.5825 - val_loss: 0.5083
Epoch 2/3
25/25 [==============================] - 146s 6s/step - loss: 0.3621 - val_loss: 0.3468
Epoch 3/3
25/25 [==============================] - 134s 5s/step - loss: 0.2525 - val_loss: 0.3018


In [37]:
from sklearn.metrics import classification_report
y_pred = model.predict(tf_validation_set).logits.argmax(axis=-1)
y_true = [example['labels'] for example in test_adf_tok]
report = classification_report(y_true, y_pred, target_names=['NotAffirmed', 'Affirmed'])

print(report)

27/27 [==============================] - 31s 996ms/step
              precision    recall  f1-score   support

 NotAffirmed       0.93      0.90      0.91       220
    Affirmed       0.90      0.92      0.91       204

    accuracy                           0.91       424
   macro avg       0.91      0.91      0.91       424
weighted avg       0.91      0.91      0.91       424

